## Test per progetto AI

E' necessario caricare i file roads_small.json e nodes_small.json

In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [3]:
nodes = json.load(open('new_nodes.json', 'r'))
roads = json.load(open('new_roads.json', 'r'))
print("Num nodes", len(nodes))
print("Num roads", len(roads))

Num nodes 2341
Num roads 10325


Create two maps as lookup tables for the nodes

In [4]:
#create a lookup table for point coordinates
node_coords = {point['id']:tuple(point['coordinates']) for point in nodes} #dictonary initialization with comprehension
coords_node = {tuple(point['coordinates']):point['id'] for point in nodes} #tuples are hashable-> can be key to a dictionary

In [5]:
print(node_coords[roads[11]['p1']]) #retrieve coordinates from node ID

(12.27, 45.46)


A questo punto bisogna generare le query degli utenti:

semplificazione: l'orario non influisce sui tempi di percorrenza -> dipendono solo dalla distanza

Consider each trip as starting or ending in one of the nodes: the function below serves to approximate the closest node on the map

Regular transfer (not related to pendolarism) is modeled as a gaussian random variable with respect to location ($\mu = 15km, \sigma = 10km$)  

In [6]:
def generate_ratings(n_driver):
  mean, std = 3.5, 0.5
  x = np.random.normal(mean, std, size=(n_driver, 1))
  ratings = np.empty(shape = (n_driver, 1))
  for i in range(n_driver):
    y = math.ceil(x[i])
    if (y - x[i] > 0.5) :
      ratings[i] = y - 0.5
    else :
      ratings[i] = y
      
  return ratings


In [7]:
# pool of end points 
# define a random number of destinations (e.g. from 8 to 12)
num_dest = random.randrange(8, 12, 1)
pool_dest = []
for i in range(num_dest):
  rand = random.randrange(len(node_coords))
  id_dest = list(node_coords.keys())[rand]
  coord_dest = list(node_coords.values())[rand]
  pool_dest.append({'id':id_dest, 'coordinates':coord_dest})

print(pool_dest)

[{'id': 358, 'coordinates': (11.99, 45.47)}, {'id': 1836, 'coordinates': (11.84, 45.22)}, {'id': 4, 'coordinates': (12.68, 45.51)}, {'id': 2068, 'coordinates': (11.91, 45.34)}, {'id': 1090, 'coordinates': (12.08, 45.4)}, {'id': 327, 'coordinates': (12.12, 45.26)}, {'id': 2175, 'coordinates': (12.15, 45.2)}, {'id': 799, 'coordinates': (11.8, 45.45)}, {'id': 1537, 'coordinates': (12.12, 45.48)}, {'id': 1723, 'coordinates': (11.86, 45.49)}]


In [8]:
def generate_trips(n):
  # needs to define, for each driver/rider, one random starting point, and peak from the pool of destinations a random end points
  trips_start = []
  trips_dest = []
  for i in range(n):
    # start point
    rand_s = random.randrange((len(node_coords)))
    id_start = list(node_coords.keys())[rand_s]
    coord_start = list(node_coords.values())[rand_s]
    trips_start.append({'id':id_start, 'coordinates':coord_start})
    # end point
    rand_d = random.randrange(num_dest)
    dest = pool_dest[rand_d]
    trips_dest.append(dest)

  return trips_start, trips_dest


In [9]:
def generate_schedules(n):
  #time probability mass: how many requests per 2-hour slot in percentage
  time_prob = {   6 : 0.2,
                  8 : 0.1,
                  10 : 0.07,
                  12 : 0.1,
                  14 : 0.07,
                  16 : 0.15,
                  18 : 0.15,
                  20 : 0.05,
                  22 : 0.05,
                  0 : 0.01,
                  2 : 0.01,
                  4 : 0.04 }

  times = []
  hours = list(time_prob.keys())
  prob = list(time_prob.values())
  for i in range(n):
    times.append(str(random.choices(hours, prob)[0]) + ":00")

  return times

In [10]:
class Driver:
  def __init__(self, id, rating, hour, start_point, end_point):
    self.id = id
    self.rating = rating
    self.hour = hour
    self.start_point = start_point
    self.end_point = end_point

  def __str__(self):
    return "Driver ( id = " + str(self.id) + ", rating = " + str(self.rating) + ", hour = " + self.hour + ", start = " + str(self.start_point) + ", dest = " + str(self.end_point) + ")"

class Rider:
  def __init__(self, id, hour, start_point, end_point):
    self.id = id
    self.hour = hour
    self.start_point = start_point
    self.end_point = end_point

  def __str__(self):
    return "Rider ( id = " + str(self.id) + ", hour = " + self.hour + ", start = " + str(self.start_point) + ", dest = " + str(self.end_point) + ")"

In [11]:
def generate_data():
  # 1- generate n_drivers and n_riders
  # range: start, end, stepsize
  n_drivers = random.randrange(40, 50, 1)
  n_riders = random.randrange(90, 150, 15)
  print("N_drivers: ", n_drivers)
  print("N_riders: ", n_riders)
  # 2 - generate start point and end point for drivers and riders
  trips_start_drivers, trips_dest_drivers = generate_trips(n_drivers)
  trips_start_riders, trips_dest_riders = generate_trips(n_riders)
  # 3 - generate schedules
  drivers_schedule = generate_schedules(n_drivers)
  riders_schedule = generate_schedules(n_riders)
  # 4- generate ratings for drivers
  ratings = generate_ratings(n_drivers)
  # 5 - parse in a list of objects
  riders = []
  drivers = []
  for i in range(n_riders):
    riders.append( Rider(i, riders_schedule[i], trips_start_riders[i], trips_dest_riders[i]))

  for i in range(n_drivers):
    drivers.append( Driver(i, ratings[i][0], drivers_schedule[i], trips_start_drivers[i], trips_dest_drivers[i]))

  return drivers, riders



In [12]:
drivers, riders = generate_data()

for i in range(5) :
  print(drivers[i])

print("\n\n***********************************************************************\n\n")

for i in range(5) :
  print(riders[i])


N_drivers:  42
N_riders:  105
Driver ( id = 0, rating = 3.5, hour = 16:00, start = {'id': 1312, 'coordinates': (12.7, 45.56)}, dest = {'id': 799, 'coordinates': (11.8, 45.45)})
Driver ( id = 1, rating = 4.0, hour = 14:00, start = {'id': 212, 'coordinates': (11.87, 45.4)}, dest = {'id': 1537, 'coordinates': (12.12, 45.48)})
Driver ( id = 2, rating = 3.0, hour = 6:00, start = {'id': 1540, 'coordinates': (11.75, 45.24)}, dest = {'id': 327, 'coordinates': (12.12, 45.26)})
Driver ( id = 3, rating = 4.5, hour = 18:00, start = {'id': 472, 'coordinates': (11.94, 45.4)}, dest = {'id': 1537, 'coordinates': (12.12, 45.48)})
Driver ( id = 4, rating = 4.0, hour = 6:00, start = {'id': 1182, 'coordinates': (12.05, 45.59)}, dest = {'id': 1537, 'coordinates': (12.12, 45.48)})


***********************************************************************


Rider ( id = 0, hour = 20:00, start = {'id': 1714, 'coordinates': (11.75, 45.32)}, dest = {'id': 1090, 'coordinates': (12.08, 45.4)})
Rider ( id = 1, hour

In [14]:
# create file input.dat
f = open("input.dat", "w")
content = "DIMENSION : " + str(len(nodes)) + "\n"
content += "DRIVERS : " + str(len(drivers)) + "\n"
content += "RIDERS : " + str(len(riders)) + "\n"
content += "NODE_COORD_SECTION\n"
for node in nodes :
  content += str(node.get("id")) + " " + str(node.get("coordinates")[0]) + " " + str(node.get("coordinates")[1]) + "\n"
content += "ROADS_SECTION\n"
for road in roads :
  content += str(road.get("p1")) + " " + str(road.get("p2")) + "\n"
content += "DRIVERS_STARTING_COORD\n"
for driver in drivers:
  content += str(driver.start_point.get("id")) + " " + str(driver.start_point.get("coordinates")[0]) + " " + str(driver.start_point.get("coordinates")[1]) + "\n"
content += "DRIVERS_DESTINATION_COORD\n"
for driver in drivers:
  content += str(driver.end_point.get("id")) + " " + str(driver.end_point.get("coordinates")[0]) + " " + str(driver.end_point.get("coordinates")[1]) + "\n"
content += "RIDERS_STARTING_COORD\n"
for rider in riders:
  content += str(rider.start_point.get("id")) + " " + str(rider.start_point.get("coordinates")[0]) + " " + str(rider.start_point.get("coordinates")[1]) + "\n"
content += "RIDERS_DESTINATION_COORD\n"
for rider in riders:
  content += str(rider.end_point.get("id")) + " " + str(rider.end_point.get("coordinates")[0]) + " " + str(rider.end_point.get("coordinates")[1]) + "\n"
content += "EOF"
f.write(content)
f.close()